In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pycountry
import csv

conn = sqlite3.connect('./data/countryDatabase.db')
cur = conn.cursor()

# Create all the tables
The below code will create all the tables that correspond to the relevant csv files. 

The time range for this data is 2000-2022 on account of data availability.

The SQL files are not present in the data, but if you run all the code, the SQL will be auto generated in the `data` folder.

In [2]:
# greenhouse gas data 
# units are in millions of metric tons of greenhouse gases
cur.execute("CREATE TABLE greenhouseGas(ISO_country)")

for year in range(2000, 2022 + 1):
    conn.execute('ALTER TABLE greenhouseGas ADD COLUMN "{}" FLOAT'.format(year))

In [3]:
# military spending data 
# units are in percentage of GDP
cur.execute("CREATE TABLE militarySpending(ISO_country)")

for year in range(2000, 2022 + 1):
    conn.execute('ALTER TABLE militarySpending ADD COLUMN "{}" FLOAT'.format(year))

In [4]:
# military spending data
# units are in USD
cur.execute("CREATE TABLE militarySpendingUSD(ISO_country)")

for year in range(2000, 2022 + 1):
    conn.execute('ALTER TABLE militarySpendingUSD ADD COLUMN "{}" FLOAT'.format(year))

In [5]:
# population data
cur.execute("CREATE TABLE population(ISO_country)")

for year in range(2000, 2022 + 1):
    conn.execute('ALTER TABLE population ADD COLUMN "{}" FLOAT'.format(year))

# Preparing & Cleaning Data
The below code will clean the data into a proper format and prepare it for insertion into the SQL tables.

In [6]:
# greenhouse gas data
# change the below file path to fit your file structure
ccDF = pd.read_csv('./data/greenhouse_gas_data.csv', sep=';')
ccDF = ccDF.drop(['Country', 'ISO2', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'ObjectId', 'Industry', 'F2023', 'F2024', 'F2025', 'F2026', 'F2027', 'F2028', 'F2029', 'F2030'], axis=1) # remove irrelevant columns
for col in range(1970, 2000): # drop columns that are outside time range (2000-2022)
    ccDF = ccDF.drop(f"F{col}", axis=1)
for col in range(2000, 2023): # rename columns from format "F2000" to "2000"
    ccDF = ccDF.rename(columns={f"F{col}": str(col)})
# the dataset contains data for various different kinds of gases, we're interested in the aggregate that is labelled as "greenhouse gasses"
# keep only greenhouse gas net data
ccDF = ccDF[(ccDF['Indicator'] == 'Total GHG emissions including land-use, land-use change and forestry') & (ccDF['Gas_Type'] == 'Greenhouse gas')]
ccDF = ccDF.drop(['Indicator', 'Gas_Type', 'Unit', 'Scale'], axis=1)
ccDF = ccDF.rename(columns={"ISO3": "ISO_country"})
ccDF.reset_index(inplace=True)
ccDF.drop(columns=['index'], inplace=True)
ccDF = ccDF[(ccDF['ISO_country'] != 'NA605') & (ccDF['ISO_country'] != 'OCETMP')]
ccDF = ccDF.sort_values('2022', ascending=False) # sort in descending order of 2022

# print length of data, columns and head to check that it is all correct
print(f"Length of dataframe: {len(ccDF.index)}")
print(ccDF.columns)
ccDF.head(10)



Length of dataframe: 209
Index(['ISO_country', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')


,ISO_country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
40,CHN,5079.996777,5295.214492,5629.884873,6349.302251,7227.178765,8102.437217,8902.651599,9515.877756,9741.563585,...,12777.858230,12942.777130,12772.459600,12799.810270,13062.499690,13649.331320,13958.930190,14232.311930,14985.606710,15037.299140
202,USA,6525.394520,6406.295840,6487.102239,6517.858735,6739.835223,6685.153433,6581.822659,6727.961642,6524.704979,...,6085.387589,6124.910997,6052.126607,5758.076490,5779.375022,5982.678836,5906.576649,5238.855260,5574.159784,5705.461724
89,IND,1698.776935,1949.562561,1978.908675,2033.684184,2139.614557,2210.073028,2315.079715,2461.312367,2572.199557,...,3122.521147,3296.155226,3327.305337,3412.413105,3561.890963,3725.084963,3699.008400,3485.249112,3719.871227,3908.064862
26,BRA,1976.481688,2057.138838,2090.928084,2122.145720,2174.697827,2186.993494,2185.128555,2215.189862,2230.379749,...,1553.014360,1592.007730,1582.368553,1679.764116,1698.102046,1664.058627,1679.018165,1682.619816,1758.302512,1741.006778
157,RUS,1406.480603,1389.507593,1342.812786,1369.443428,1394.473047,1414.500243,1472.141210,1453.499096,1444.594104,...,1417.626201,1366.815483,1433.899965,1404.472045,1464.529719,1548.254793,1569.302544,1489.256273,1631.684988,1608.969791
90,IDN,1065.281135,904.184261,1411.476619,1079.155591,1446.310373,1176.670769,1620.146413,1057.492559,1054.709044,...,1685.539620,2075.511010,2124.513977,1472.516821,1501.118163,1776.238704,2066.570888,1605.091840,1438.155277,1433.148620
98,JPN,1288.444851,1262.069820,1284.366020,1279.689455,1274.503007,1287.941781,1271.457581,1310.764900,1247.783006,...,1340.222469,1294.318582,1261.263057,1247.434831,1230.071926,1186.494825,1156.633780,1092.931404,1115.993092,1123.624162
91,IRN,505.186889,487.684249,507.005615,556.022628,596.384497,639.288094,687.656961,720.054224,726.696879,...,833.512844,858.441488,857.021547,904.905109,937.980030,950.886170,915.886881,901.613634,936.625753,952.044593
125,MEX,662.224390,567.650045,578.050954,608.397681,617.041183,643.622055,668.238364,682.584696,692.541293,...,809.487688,795.877777,805.438928,814.874440,819.306113,796.229438,808.125513,756.060201,781.875150,836.004316
163,SAU,355.024041,367.948642,388.571077,414.769908,437.923904,466.493645,491.246215,513.948661,551.765287,...,696.018485,739.747159,771.903726,777.262898,774.572566,758.472592,761.485078,755.368856,779.973162,810.511607


In [7]:
# population data
# change the below file path to fit your file structure
popDF = pd.read_csv('./data/un_demographics.csv')
popDF = popDF.dropna(subset=['ISO3_code'])

popDF = popDF[['ISO3_code', 'Time', 'TPopulation1Jan']] # only select the total population on the first of january on a year
popDF = popDF.pivot_table(index='ISO3_code', columns='Time', values='TPopulation1Jan') # reshape the dataframe to have years as columns
for year in range(1950, 2102): # drop all columns not within our time range (this dataset also contains projections up to 2102)
    if year >= 2000 and year <= 2022:
        pass
    else:
        popDF = popDF.drop(year, axis=1)
popDF.reset_index(inplace=True)
popDF.rename(columns={'ISO3_code':'ISO_country'}, inplace=True)
popDF = popDF.rename(
    {2000: '2000', 2001: '2001', 2002: '2002', 2003: '2003', 2004: '2004', 2005: '2005', 2006: '2006', 2007: '2007', 2008: '2008', 2008: '2008', 2009: '2009', 2010: '2010', 2011: '2011', 2012: '2012', 2013: '2013', 2014: '2014', 2015: '2015', 2016: '2016', 2017: '2017', 2018: '2018', 2019: '2019', 2020: '2020', 2021: '2021', 2022: '2022'}
    , axis=1)
popDF = popDF.sort_values('2022', ascending=False) # sort in descending order of 2022
popDF[popDF.select_dtypes(include=['number']).columns] *= 1000
popDF = popDF.round(0) # round all values to 0 decimal places



# print length of data, columns and head to check that it is all correct
print(f"Length of dataframe: {len(popDF.index)}")
print(popDF.columns)
popDF.head(10)

Length of dataframe: 237
Index(['ISO_country', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object', name='Time')


/var/folders/zp/r_synnss3dnc2lgwfbyr07h80000gn/T/ipykernel_18420/1804657599.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  popDF = pd.read_csv('./data/un_demographics.csv')


Time,ISO_country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
38,CHN,1.259610e+09,1.268588e+09,1.276891e+09,1.284961e+09,1.292786e+09,1.300848e+09,1.308927e+09,1.317246e+09,1.325781e+09,...,1.371616e+09,1.380585e+09,1.389795e+09,1.397636e+09,1.406143e+09,1.414409e+09,1.419730e+09,1.423998e+09,1.425862e+09,1.425925e+09
97,IND,1.050012e+09,1.069256e+09,1.088686e+09,1.107940e+09,1.126890e+09,1.145639e+09,1.163639e+09,1.181109e+09,1.198275e+09,...,1.282874e+09,1.299390e+09,1.315103e+09,1.330630e+09,1.346642e+09,1.361749e+09,1.376258e+09,1.389966e+09,1.402808e+09,1.412320e+09
221,USA,2.808166e+08,2.839805e+08,2.869605e+08,2.897400e+08,2.924796e+08,2.954161e+08,2.982692e+08,3.012370e+08,3.042498e+08,...,3.180311e+08,3.207193e+08,3.233486e+08,3.258669e+08,3.285535e+08,3.310290e+08,3.332511e+08,3.353882e+08,3.364958e+08,3.374995e+08
95,IDN,2.125441e+08,2.156007e+08,2.186241e+08,2.216060e+08,2.245542e+08,2.273230e+08,2.302873e+08,2.333076e+08,2.364090e+08,...,2.517798e+08,2.547720e+08,2.576875e+08,2.604964e+08,2.632039e+08,2.657938e+08,2.683399e+08,2.708258e+08,2.728901e+08,2.746163e+08
163,PAK,1.518225e+08,1.569174e+08,1.615181e+08,1.650075e+08,1.687458e+08,1.725514e+08,1.761928e+08,1.799472e+08,1.839019e+08,...,2.038308e+08,2.068443e+08,2.096590e+08,2.122796e+08,2.147700e+08,2.179893e+08,2.214737e+08,2.251129e+08,2.292806e+08,2.335236e+08
154,NGA,1.212355e+08,1.244685e+08,1.278369e+08,1.313292e+08,1.349104e+08,1.386033e+08,1.423782e+08,1.462814e+08,1.503067e+08,...,1.723969e+08,1.770554e+08,1.817027e+08,1.862889e+08,1.910450e+08,1.959469e+08,2.008284e+08,2.057806e+08,2.108742e+08,2.159284e+08
29,BRA,1.746927e+08,1.770548e+08,1.793690e+08,1.815844e+08,1.836742e+08,1.857699e+08,1.878247e+08,1.898166e+08,1.917423e+08,...,2.008590e+08,2.025845e+08,2.043348e+08,2.060416e+08,2.076776e+08,2.093324e+08,2.110008e+08,2.125649e+08,2.138277e+08,2.148248e+08
19,BGD,1.279756e+08,1.304111e+08,1.329299e+08,1.353498e+08,1.376566e+08,1.399228e+08,1.419024e+08,1.433553e+08,1.449166e+08,...,1.530540e+08,1.550063e+08,1.569163e+08,1.587437e+08,1.608255e+08,1.627625e+08,1.646055e+08,1.664270e+08,1.684149e+08,1.702976e+08
179,RUS,1.471298e+08,1.465599e+08,1.459111e+08,1.452691e+08,1.446243e+08,1.440830e+08,1.435171e+08,1.431602e+08,1.430752e+08,...,1.437855e+08,1.441283e+08,1.444419e+08,1.448949e+08,1.453234e+08,1.455817e+08,1.457229e+08,1.457617e+08,1.454730e+08,1.447325e+08
134,MEX,9.711212e+07,9.863476e+07,1.001538e+08,1.016803e+08,1.031783e+08,1.047133e+08,1.061715e+08,1.076021e+08,1.090039e+08,...,1.165419e+08,1.180395e+08,1.194723e+08,1.208275e+08,1.222110e+08,1.234676e+08,1.245602e+08,1.256105e+08,1.263861e+08,1.270241e+08


In [8]:
# this function gets a country name and turns it into the ISO3 standard. It can then be mapped onto the dataframe.

def country_to_iso(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None
    
# military spending data (GDP%)
# change the below file path to fit your file structure
milDF = pd.read_csv('./data/sipri_gdp_2000-2023.csv')
milDF = milDF.replace('...', None) # recode NaNs
cols = milDF.columns
milDF.iloc[:, 1:] = milDF.iloc[:, 1:].applymap(lambda x: x.replace('%', '') if x is not None else x) # remove % signs
milDF[cols[1:]] = milDF[cols[1:]].apply(pd.to_numeric, errors='coerce')
milDF = milDF.interpolate(method='linear', axis=0, limit_direction="forward") # interpolate the missing values
milDF['ISO_country'] = milDF['Country'].apply(country_to_iso)
milDF = milDF.drop('Country', axis=1)
milDF = milDF.sort_values('2022', ascending=False) # sort in descending order of 2022

# print length of data, columns and head to check that it is all correct
print(f"Length of dataframe: {len(milDF.index)}")
print(milDF.columns)
milDF.head(10)



Length of dataframe: 174
Index(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', 'ISO_country'],
      dtype='object')


,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,ISO_country
135,2.150,1.890,1.980,2.120,1.97,2.41,2.390,2.470,2.300,1.91,...,2.25,3.250,3.150,2.880,3.190,3.520,3.810,3.23,33.55,UKR
136,1.565,1.400,1.430,1.515,1.43,1.63,1.590,1.685,1.585,1.37,...,1.50,1.975,1.940,1.820,1.970,2.135,2.345,2.05,17.16,None
168,10.530,11.420,9.760,8.690,8.08,7.73,7.850,8.530,7.350,9.62,...,10.68,13.330,9.870,10.220,9.140,8.130,9.180,7.58,7.42,SAU
167,8.810,9.810,3.930,3.330,2.43,1.99,1.750,1.960,2.010,1.99,...,9.77,11.450,10.215,9.315,8.705,7.785,8.605,6.45,6.96,QAT
47,1.625,1.565,1.465,1.370,1.41,1.46,1.145,1.075,1.720,1.64,...,1.82,1.670,1.840,1.870,1.960,3.150,2.840,3.11,5.44,TGO
166,7.090,8.200,8.100,7.960,7.86,7.71,7.100,6.720,4.960,6.07,...,8.86,9.570,10.560,8.410,8.270,7.440,8.030,6.56,5.17,OMN
165,5.390,5.450,4.750,4.680,4.51,4.48,4.580,4.630,4.000,4.02,...,4.68,4.490,5.060,4.510,4.850,4.240,3.030,3.37,4.85,LBN
163,6.260,5.890,5.450,6.000,5.14,4.80,4.660,5.850,5.990,6.39,...,4.20,4.190,4.440,4.680,4.560,4.570,4.700,4.81,4.84,JOR
0,3.430,3.820,3.700,3.250,3.28,2.83,2.640,2.920,3.020,3.85,...,5.55,6.270,6.380,5.920,5.480,6.000,6.690,5.59,4.78,DZA
130,2.270,2.310,2.240,2.430,2.63,2.30,3.420,2.860,3.290,3.33,...,4.56,5.460,3.690,3.760,3.560,3.870,5.390,5.27,4.55,AZE


In [9]:
# military spending data (USD)
# change the below file path to fit your file structure
milDF_USD = pd.read_csv('./data/sipri_2022usd_data_2000-2023.csv')
milDF_USD = milDF_USD.replace('...', None)
cols = milDF_USD.columns
milDF_USD[cols[1:]] = milDF_USD[cols[1:]].apply(pd.to_numeric, errors='coerce')
milDF_USD = milDF_USD.interpolate(method='linear', axis=0, limit_direction="forward") # interpolate the missing values 
milDF_USD['ISO_country'] = milDF_USD['Country'].apply(country_to_iso)
milDF_USD = milDF_USD.drop('Country', axis=1)
milDF_USD = milDF_USD.sort_values('2022', ascending=False) # sort in descending order of 2022

# print length of data, columns and head to check that it is all correct
print(f"Length of dataframe: {len(milDF_USD.index)}")
print(milDF_USD.columns)
milDF_USD.head(10)

Length of dataframe: 174
Index(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', 'ISO_country'],
      dtype='object')


,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,ISO_country
65,503680.10,507770.80,570129.60,648901.60,707251.00,739826.20,750486.90,770512.7,826562.7,891604.50,...,741463.5,724625.9,722402.3,714959.10,736477.50,778326.9,814965.20,806230.20,811591.2,USA
87,45586.10,54052.90,62082.20,67187.90,74005.60,81267.80,93544.50,102829.8,112507.1,136520.50,...,197998.8,213542.4,225835.1,239724.30,253766.40,266129.3,278673.30,285930.50,297999.0,CHN
83,30792.60,31865.10,31766.50,32475.70,37723.50,40146.30,40471.50,40960.4,46459.4,54698.70,...,57867.1,58416.9,64374.7,68909.50,71336.20,76372.3,76835.60,76348.50,80955.8,IND
168,32860.60,35001.70,30722.90,30941.30,34334.00,41461.50,47230.60,54403.8,53395.8,54873.50,...,88476.4,94375.2,67526.2,75291.70,77882.10,69685.9,66536.00,63194.70,73041.5,SAU
134,22839.80,24683.00,27343.60,28670.80,29967.10,34054.90,37693.60,41029.6,45082.3,47299.80,...,67075.1,72293.5,77501.7,62801.20,60435.00,63143.9,64627.40,65907.70,71981.1,RUS
156,54375.00,56528.50,59889.10,64153.20,64949.30,65556.50,65857.20,67650.2,70611.9,71743.30,...,62696.4,60286.3,60076.1,60185.90,60638.90,63157.8,64161.10,67500.70,69998.7,GBR
143,46431.40,45668.20,45792.40,45166.60,43783.00,36489.50,42138.70,42203.2,43201.0,44837.10,...,43670.6,44420.5,46348.1,47677.50,48993.20,53742.5,57056.10,56513.10,57807.7,DEU
142,48554.40,48404.70,49399.20,50889.30,52278.60,51207.10,51443.00,51423.6,51049.9,54826.90,...,50435.8,51815.5,53824.5,54532.60,53187.30,54066.0,55623.20,56647.00,56999.7,FRA
88,45779.80,46581.20,46813.20,46872.40,46685.20,46600.80,46006.90,45408.9,44971.9,45814.40,...,46335.4,47149.4,46869.3,46559.20,48903.00,50293.9,49862.80,50957.50,53946.5,JPN
89,33963.55,34680.95,35110.85,35538.35,35976.45,36964.25,37159.75,37501.9,38251.4,39663.85,...,41975.1,43112.7,43462.2,43675.55,45881.85,48303.7,49306.55,50915.65,51782.4,None


# Inserting Data

In [10]:
tables = ['greenhouseGas', 'militarySpending', 'militarySpendingUSD', 'population']
dataframes = [ccDF, milDF, milDF_USD, popDF]

for index, table in enumerate(tables):
    tableName = table
    dataframe = dataframes[index]
    for index, row in dataframe.iterrows():
        ISO_country = row['ISO_country']
        cur.execute(f"INSERT INTO {tableName} VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (ISO_country, row['2000'], row['2001'], row['2002'], row['2003'], row['2004'], row['2005'], row['2006'], row['2007'], row['2008'], row['2009'], row['2010'], row['2011'], row['2012'], row['2013'], row['2014'], row['2015'], row['2016'], row['2017'], row['2018'], row['2019'], row['2020'], row['2021'], row['2022']))

In [11]:
conn.commit()
conn.close()

# Visualising Data
The visualisations are not yet complete.


## Maps

In [12]:
annex1 = ['AUS', 'AUT', 'BLR', 'BEL', 'BGR', 'CAN', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'EUU', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ITA', 'JPN', 'LVA', 'LIE', 'LTU', 'LUX', 'MLT', 'MCO', 'NLD', 'NZL', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'TUR', 'UKR', 'GBR', 'USA']


In [13]:
emissionsCapitaDF =  ccDF.set_index('ISO_country') / popDF.set_index('ISO_country')
emissionsCapitaDF.reset_index(inplace=True)


emissionsCapitaDF = emissionsCapitaDF.interpolate()
emissionsCapitaDF = emissionsCapitaDF.sort_values(by='2022', ascending=False)


In [14]:
emissionsCapitaDF = emissionsCapitaDF[emissionsCapitaDF['ISO_country'].isin(annex1)]
melted_df = emissionsCapitaDF.melt(id_vars=['ISO_country'], var_name='Year', value_name='Value')
melted_df = melted_df.drop(melted_df[melted_df['ISO_country'] == 'FLK'].index)
import plotly.express as px

fig = px.line(melted_df, x='Year', y='Value', color='ISO_country')
fig.show()



In [15]:
ccWorld = pd.DataFrame(ccDF.sum(axis=0)).T
ccWorld['ISO_country'] = 'World'
ccWorld = ccWorld.reset_index(drop=True)
ccWorld_melted = ccWorld.melt(id_vars=['ISO_country'], var_name='Year', value_name='Value')
# y - millions of metric tons of GHG
fig = px.line(ccWorld_melted, x='Year', y='Value', color='ISO_country')
fig.show()



In [16]:
popWorld = pd.DataFrame(popDF.sum(axis=0)).T
popWorld['ISO_country'] = 'World'
popWorld = popWorld.reset_index(drop=True)
popWorld_melted = popWorld.melt(id_vars=['ISO_country'], var_name='Year', value_name='Value')
fig = px.line(popWorld_melted, x='Year', y='Value', color='ISO_country')
fig.show()


In [17]:
ccCapitaDF = ccWorld.set_index('ISO_country') * 1000000 / popWorld.set_index('ISO_country')
ccCapitaDF = ccCapitaDF.reset_index()
ccCapitaDF.head()

,ISO_country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,World,6.035629,5.984896,6.09916,6.198919,6.440622,6.517651,6.671549,6.703227,6.640824,...,6.762506,6.781445,6.712093,6.599974,6.637687,6.739539,6.723199,6.384985,6.599143,6.61156


In [18]:
ccCapita_melted = ccCapitaDF.melt(id_vars=['ISO_country'], var_name='Year', value_name='Value')
fig = px.line(ccCapita_melted, x='Year', y='Value', color='ISO_country')
fig.show()
